In [1]:
import argparse

import torch
import torch.nn.functional as F
import numpy as np
from src.args import ArgumentParserRGBDSegmentation
from src.build_model import build_model
# from src.confusion_matrix import ConfusionMatrixTensorflow
from src.prepare_data import prepare_data
import pickle
from sens_reader import scannet_scene_reader
from matplotlib import pyplot as plt
import os

In [3]:
help(F.interpolate)

Help on function interpolate in module torch.nn.functional:

interpolate(input, size=None, scale_factor=None, mode='nearest', align_corners=None, recompute_scale_factor=None)
    Down/up samples the input to either the given :attr:`size` or the given
    :attr:`scale_factor`
    
    The algorithm used for interpolation is determined by :attr:`mode`.
    
    Currently temporal, spatial and volumetric sampling are supported, i.e.
    expected inputs are 3-D, 4-D or 5-D in shape.
    
    The input dimensions are interpreted in the form:
    `mini-batch x channels x [optional depth] x [optional height] x width`.
    
    The modes available for resizing are: `nearest`, `linear` (3D-only),
    `bilinear`, `bicubic` (4D-only), `trilinear` (5D-only), `area`
    
    Args:
        input (Tensor): the input tensor
        size (int or Tuple[int] or Tuple[int, int] or Tuple[int, int, int]):
            output spatial size.
        scale_factor (float or Tuple[float]): multiplier for spatial s

In [2]:
args = pickle.load(open('args.p','rb'))
model, device = build_model(args, n_classes=40)

checkpoint = torch.load(args.ckpt_path,
                        map_location=lambda storage, loc: storage)
model.load_state_dict(checkpoint['state_dict'])
model.eval()
model.to(device)

/home/motion/Instance Retrieval/ESANet/src/build_model.py:29: UserWarning: Argument --channels_decoder is ignored when --decoder_chanels_mode decreasing is set.
  warnings.warn('Argument --channels_decoder is ignored when '
/home/motion/Instance Retrieval/ESANet/src/models/resnet.py:101: UserWarning: parameters groups, base_width and norm_layer are ignored in NonBottleneck1D
  warnings.warn('parameters groups, base_width and norm_layer are '
/home/motion/Instance Retrieval/ESANet/src/models/model.py:163: UserWarning: for the context module the learned upsampling is not possible as the feature maps are not upscaled by the factor 2. We will use nearest neighbor instead.
  warnings.warn('for the context module the learned upsampling is '


Device: cuda:0


ESANet(
  (activation): ReLU(inplace=True)
  (encoder_rgb): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): NonBottleneck1D(
        (conv3x1_1): Conv2d(64, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        (conv1x3_1): Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (bn1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
        (conv3x1_2): Conv2d(64, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        (conv1x3_2): Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (bn2): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
     

In [ ]:
lim= 100
root_dir = "/home/motion/data/scannet_v2"
my_ds = scannet_scene_reader(root_dir, 'scene0050_00',lim = lim)
tmp = my_ds[0]

In [ ]:
dataset, preprocessor = prepare_data(args, with_input_orig=True)
import time

In [ ]:
%time
for i in range(0,len(my_ds),30):
    start = time.time()
    tmp = my_ds[i]
    img_rgb = tmp['color']
    img_depth = tmp['depth']
    h, w, _ = img_rgb.shape

    # preprocess sample
    sample = preprocessor({'image': img_rgb, 'depth': img_depth})

    # add batch axis and copy to device
    image = sample['image'][None].to(device)
    depth = sample['depth'][None].to(device)

    h, w, _ = img_rgb.shape

    # preprocess sample
    sample = preprocessor({'image': img_rgb, 'depth': img_depth})

    # add batch axis and copy to device
    image = sample['image'][None].to(device)
    depth = sample['depth'][None].to(device)

    # apply network
    pred = model(image, depth)
#     print(pred.shape)
#     pred = F.interpolate(pred, (h, w),
#                          mode='bilinear', align_corners=False)
    pred = torch.argmax(pred, dim=1)
    pred = pred.cpu().numpy().squeeze().astype(np.uint8)
#     print('took {}'.format(time.time()-start))
#     # show result
    pred_colored = dataset.color_label(pred, with_void=False)
    fig, axs = plt.subplots(1, 3, figsize=(16, 3))
    [ax.set_axis_off() for ax in axs.ravel()]
    axs[0].imshow(img_rgb)
    axs[1].imshow(img_depth, cmap='gray')
    axs[2].imshow(pred_colored)

    plt.suptitle(f"Image")
    # plt.savefig('./result.jpg', dpi=150)
    plt.show()
    break

In [ ]:
h, w, _ = img_rgb.shape

# preprocess sample
sample = preprocessor({'image': img_rgb, 'depth': img_depth})

# add batch axis and copy to device
image = sample['image'][None].to(device)
depth = sample['depth'][None].to(device)

# apply network
pred = model(image, depth)
# pred = F.interpolate(pred, (h, w),
#                      mode='bilinear', align_corners=False)
# pred = torch.argmax(pred, dim=1)
# pred = pred.cpu().numpy().squeeze().astype(np.uint8)

# # show result
# pred_colored = dataset.color_label(pred, with_void=False)
# fig, axs = plt.subplots(1, 3, figsize=(16, 3))
# [ax.set_axis_off() for ax in axs.ravel()]
# axs[0].imshow(img_rgb)
# axs[1].imshow(img_depth, cmap='gray')
# axs[2].imshow(pred_colored)

# plt.suptitle(f"Image")
# # plt.savefig('./result.jpg', dpi=150)
# plt.show()

In [ ]:
from PIL import Image

import os, struct
import numpy as np
import zlib
import imageio.v2 as imageio
import cv2
import csv
import shutil
from tqdm import tqdm
import zipfile
import pandas as pd


root_dir = "/home/motion/data/scannet_v2"


label_file = os.path.join(root_dir, 'scannetv2-labels.combined.tsv')
scannet_id_nyu_dict = {}

with open(label_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row_dict in reader:
        scannet_id = row_dict['id']
        nyu40_id = row_dict['nyu40id']
        scannet_id_nyu_dict[int(scannet_id)] = int(nyu40_id)

# # map label as instructed in http://kaldir.vc.in.tum.de/scannet_benchmark/labelids.txt
# VALID_CLASS_IDS = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 24, 28, 33, 34, 36, 39])

# scannet_subset_map = np.zeros(41) # NYU40 has 40 labels
# for i in range(len(VALID_CLASS_IDS)):
#     scannet_subset_map[VALID_CLASS_IDS[i]] = i + 1

# # This dict maps from fine-grained ScanNet ids (579 categories)
# # to the 20 class subset as in the benchmark
# scannet_mapping = np.zeros(max(scannet_id_nyu_dict) + 1)

# for k in scannet_id_nyu_dict:
#     scannet_mapping[k] = scannet_subset_map[scannet_id_nyu_dict[k]]
df = pd.read_csv(label_file,sep = '\t')

subset = df.loc[:,['id','raw_category','nyu40id','nyu40class']]
subset.loc[:,'new_class'] = 0

ordered_relevant_classes = np.array([1,2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 24, 28, 33, 34,36, 39]).tolist()
a = np.zeros(41)
for i in ordered_relevant_classes:
    a[i] = ordered_relevant_classes.index(i)+1

subset.loc[subset.nyu40id.isin(ordered_relevant_classes),'new_class'] = a[subset.loc[subset.nyu40id.isin(ordered_relevant_classes),'nyu40id']]
subset.new_class = subset.new_class.astype(int)

conversion = np.zeros(subset.id.max()+1)
conversion[subset.id] = subset.new_class
conversion

In [ ]:
CLASS_NAMES_40 = ['void',
                      'wall', 'floor', 'cabinet', 'bed', 'chair', 'sofa',
                      'table', 'door', 'window', 'bookshelf', 'picture',
                      'counter', 'blinds', 'desk', 'shelves', 'curtain',
                      'dresser', 'pillow', 'mirror', 'floor mat', 'clothes',
                      'ceiling', 'books', 'refridgerator', 'television',
                      'paper', 'towel', 'shower curtain', 'box', 'whiteboard',
                      'person', 'night stand', 'toilet', 'sink', 'lamp',
                      'bathtub', 'bag',
                      'otherstructure', 'otherfurniture', 'otherprop']

In [ ]:
tmp = subset.loc[:,['raw_category','new_class','nyu40class']].drop_duplicates().reset_index(drop = True)
tmp = tmp.sort_values(by = 'new_class').reset_index(drop = True)
a = np.zeros(41)

for i in range(tmp.shape[0]):
    if(tmp.loc[i,'new_class']!=0):
#         print(tmp.loc[i,'raw_category'],tmp.loc[i,'new_class'],CLASS_NAMES_40.index(tmp.loc[i,'nyu40class']))
        a[CLASS_NAMES_40.index(tmp.loc[i,'nyu40class'])] = tmp.loc[i,'new_class']

In [ ]:
tmp[tmp.new_class ==5]

# testing ESANet_Loader